In [1]:
import os

In [2]:
%pwd

'c:\\Users\\lenovo\\OneDrive\\Documents\\Text_summarization_Huggingface\\research'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path  

In [6]:
from textsummarization.constants import *
from textsummarization.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [9]:
import os
from textsummarization.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

[2024-03-08 10:38:08,466: INFO : config : PyTorch version 2.2.1 available.]


In [13]:
class DataTransformation:
    def __init__(self, config= DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_example_to_feature(self,example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_length = 1024, truncation = True)

        return {
            'input_ids' : input_encoding['input_ids'],
            'attention_mask' : input_encoding['attention_mask'],
            'labels' : target_encoding['input_ids']
        }

    def convert(self):
        data_sumsum = load_from_disk(self.config.data_path) 
        data_sumsum_pt = data_sumsum.map(self.convert_example_to_feature, batched = True)
        data_sumsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))  



In [14]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-03-08 11:02:45,024: INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-03-08 11:02:45,029: INFO : common : yaml file: params.yaml loaded successfully]
[2024-03-08 11:02:45,031: INFO : common : created directory at: artifacts]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\lenovo\anaconda3\envs\testS\lib\site-packages\transformers\tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 66532.36 examples/s]
